In [1]:
import ray
from ray.rllib.agents.ppo import PPOTrainer

# Initialize Ray
ray.init(webui_host="127.0.0.1")

C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\anaconda3\envs\ai-economist\lib\site-

{'node_ip_address': '131.246.70.75',
 'raylet_ip_address': '131.246.70.75',
 'redis_address': '131.246.70.75:6379',
 'object_store_address': 'tcp://127.0.0.1:64135',
 'raylet_socket_name': 'tcp://127.0.0.1:61700',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\herrm\\AppData\\Local\\Temp\\ray\\session_2024-09-26_19-10-48_854542_15872'}

## 1. Adding an Environment Wrapper 

In [2]:
import argparse
import logging
import os
import sys
import time

from new_analysis.utils import remote, saving
from new_analysis import tf_models
import yaml
from new_analysis.env_wrapper import RLlibEnvWrapper
from ray.tune.logger import NoopLogger, pretty_print

config_path = 'new_analysis/one-step-economy/ai-economist-considerate-democracy/purely_egoistic/two-fold-tax-scheme/config.yaml' #path to config.yaml

with open(config_path, "r") as f:
        run_configuration = yaml.safe_load(f)
        
trainer_config = run_configuration.get("trainer")

# === Env ===
env_config = {
    "env_config_dict": run_configuration.get("env"),
    "num_envs_per_worker": trainer_config.get("num_envs_per_worker"),
}

# === Seed ===
if trainer_config["seed"] is None:
    try:
        start_seed = int(run_configuration["metadata"]["launch_time"])
    except KeyError:
        start_seed = int(time.time())
else:
    start_seed = int(trainer_config["seed"])

final_seed = int(start_seed % (2 ** 16)) * 1000

# === Multiagent Policies ===
dummy_env = RLlibEnvWrapper(env_config)

# Policy tuples for agent/planner policy types
agent_policy_tuple = (
    None,
    dummy_env.observation_space,
    dummy_env.action_space,
    run_configuration.get("agent_policy"),
)
planner_policy_tuple = (
    None,
    dummy_env.observation_space_pl,
    dummy_env.action_space_pl,
    run_configuration.get("planner_policy"),
)

policies = {"a": agent_policy_tuple, "p": planner_policy_tuple}

def policy_mapping_fun(i):
    if str(i).isdigit() or i == "a":
        return "a"
    return "p"

# Which policies to train
if run_configuration["general"]["train_planner"]:
    policies_to_train = ["a", "p"]
else:
    policies_to_train = ["a"]

# === Finalize and create ===
trainer_config.update(
    {
        "env_config": env_config,
        "seed": final_seed,
        "multiagent": {
            "policies": policies,
            "policies_to_train": policies_to_train,
            "policy_mapping_fn": policy_mapping_fun,
        },
        "metrics_smoothing_episodes": trainer_config.get("num_workers")
        * trainer_config.get("num_envs_per_worker"),
    }
)

def logger_creator(config):
    return NoopLogger({}, "/tmp")

trainer = PPOTrainer(
    env=RLlibEnvWrapper, config=trainer_config, logger_creator=logger_creator
)

starting_weights_path_agents = 'new_analysis/one-step-economy/ai-economist-considerate-democracy/purely_egoistic/two-fold-tax-scheme/ckpts/agent.tf.weights.global-step-1000000' # Path to starting weights for agents

saving.load_tf_model_weights(trainer, starting_weights_path_agents)
        

starting_weights_path_planner = 'new_analysis/one-step-economy/ai-economist-considerate-democracy/purely_egoistic/two-fold-tax-scheme/ckpts/planner.tf.weights.global-step-1000000' # Path to starting weights for planner
        
saving.load_tf_model_weights(trainer, starting_weights_path_planner)
        

2024-09-26 19:11:06,920	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2024-09-26 19:11:06,920	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2024-09-26 19:11:06,957	WARNING deprecation.py:30 -- DeprecationWarning: `model.custom_options` has been deprecated. Use `model.custom_model_config` instead. This will raise an error in the future!


Inside covid19_components.py: 0 GPUs are available.
No GPUs found! Running the simulation on a CPU.
(pid=9592) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9592)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=9592) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=9592)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=9592) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, 

(pid=19896) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=19896)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=19896) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=19896)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=19896) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=19896)   _np_qint

(pid=17924) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17924)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=17924) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17924)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=17924) C:\ProgramData\anaconda3\envs\ai-economist\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=17924)   _np_qint

2024-09-26 19:11:18,185	INFO trainable.py:181 -- _setup took 11.264 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2024-09-26 19:11:18,310 loaded tf model weights:
	new_analysis/one-step-economy/ai-economist-considerate-democracy/purely_egoistic/two-fold-tax-scheme/ckpts/agent.tf.weights.global-step-1000000

2024-09-26 19:11:18,448 loaded tf model weights:
	new_analysis/one-step-economy/ai-economist-considerate-democracy/purely_egoistic/two-fold-tax-scheme/ckpts/planner.tf.weights.global-step-1000000



## 3. Perform Training

And that's it! We are now ready to perform training by invoking `trainer.train()`; we call it for just a few number of iterations.

In [ ]:
NUM_ITERS = 5
for iteration in range(NUM_ITERS):
    print(f'********** Iter : {iteration} **********')
    result = trainer.train()
    print(f'''episode_reward_mean: {result.get('episode_reward_mean')}''')

By default, the results will be logged to a subdirectory of `~/ray_results`. This subdirectory will contain a file `params.json` which contains the hyperparameters, a file `result.json` which contains a training summary for each episode and a TensorBoard file that can be used to visualize training process with TensorBoard by running|
```shell
tensorboard --logdir ~/ray_results
```

## 4. Generate and Visualize the Environment's Dense Logs

At any point during training, we would also want to inspect the environment's dense logs in order to deep-dive into the training results. Introduced in our [basic tutorial](https://github.com/salesforce/ai-economist/blob/master/tutorials/economic_simulation_basic.ipynb#Visualize-using-dense-logging), dense logs are basically logs of each agent's states, actions and rewards at every point in time, along with a snapshot of the world state.

There are two equivalent ways to fetch the environment's dense logs using the trainer object.

a. Simply retrieve the dense log from the workers' environment objects

b. Generate dense log(s) from the most recent trainer policy model weights

### 4b. Generate a dense log from the most recent trainer policy model weights

We may also use the trainer object directly to play out an episode. The advantage of this approach is that we can re-sample the policy model any number of times and generate several rollouts.

In [3]:
def generate_rollout_from_current_trainer_policy(
    trainer, 
    env_obj,
    num_dense_logs=1
):
    dense_logs = {}
    for idx in range(num_dense_logs):
        # Set initial states
        agent_states = {}
        for agent_idx in range(env_obj.env.n_agents):
            agent_states[str(agent_idx)] = trainer.get_policy("a").get_initial_state()
        planner_states = trainer.get_policy("p").get_initial_state()  

        # Play out the episode
        obs = env_obj.reset(force_dense_logging=True)
        for t in range(env_obj.env.episode_length):
            actions = {}
            for agent_idx in range(env_obj.env.n_agents):
                # Use the trainer object directly to sample actions for each agent
                """
                print("Episode part: " + str(t))
                sample_list = []
                for i in range(10000):
                    sample_list.append(trainer.compute_action(
                        obs[str(agent_idx+7)], 
                        agent_states[str(agent_idx+7)], 
                        policy_id="a",
                        full_fetch=False
                    )[2]
                )
                print(np.nanmean(sample_list))
                break
                """
                
                display(obs[str(agent_idx)])
                
                actions[str(agent_idx)] = trainer.compute_action(
                    obs[str(agent_idx)], 
                    agent_states[str(agent_idx)], 
                    policy_id="a",
                    full_fetch=False
                )

            # Action sampling for the planner
            actions["p"] = trainer.compute_action(
                obs['p'], 
                planner_states, 
                policy_id='p',
                full_fetch=False
            )

            obs, rew, done, info = env_obj.step(actions)   
            if done['__all__']:
                display(env_obj.env.metrics['PeriodicTax/avg_bracket_rate/097'])
                break
        dense_logs[idx] = env_obj.env.dense_log
    return dense_logs

In [4]:
import numpy as np

dense_logs = generate_rollout_from_current_trainer_policy(
    trainer, 
    dummy_env,
    num_dense_logs=1
)

{'time': array([0.]),
 'flat': array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.01936694, 0.        , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

{'time': array([0.]),
 'flat': array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.03548354, 0.        , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

{'time': array([0.]),
 'flat': array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.05378436, 0.        , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

{'time': array([0.]),
 'flat': array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.07493671, 0.        , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

{'time': array([0.]),
 'flat': array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.10217267, 0.        , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)}

...

{'time': array([0.5]),
 'flat': array([0.5       , 0.4       , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       , 1.        ,
        0.13927247, 0.5       , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}

{'time': array([0.5]),
 'flat': array([0.5      , 0.4      , 0.       , 1.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.5      , 1.       , 0.1877788, 0.5      , 1.       ,
        0.       ], dtype=float32),
 'action_mask': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}

{'time': array([0.5]),
 'flat': array([0.5      , 0.4      , 0.       , 1.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.5      , 1.       , 0.2622324, 0.5      , 1.       ,
        0.       ], dtype=float32),
 'action_mask': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}

{'time': array([0.5]),
 'flat': array([0.5       , 0.4       , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       , 1.        ,
        0.43418166, 0.5       , 1.        , 0.        ], dtype=float32),
 'action_mask': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)}

KeyError: 'PeriodicTax/avg_bracket_rate/097'

### Visualizing results

We can use metrics and dense_logs:


There are 2 main types of logs that ```BaseEnvironment``` supports: metrics and dense logs.

**Metrics** are used to summarize an episode. Scenarios and Components each have a method for producing a metrics dictionary, which adds a tool to generate a readout on what happened. At the end of the episode, any such metrics are combined into a single metrics dictionary which is accessible through ```env.metrics```.

**Dense logs** offer a timestep-by-timestep breakdown of how an episode played out. By default, ```BaseEnvironment``` includes world state, agent state, and action info for each timestep. Components can contribute their own dense logs, which get added to the final log at the end of the episode.

Because they can be time consuming to create, dense logs are not (by default) generated during every episode. You can use the ```BaseEnvironment``` argument ```dense_log_frequency``` to set how often they are created. If, for example, you use ```dense_log_frequency=20```, then the environment will create dense logs during episodes where the number of total episode completions is a multiple of 20 (that is, every 20 episodes). If you don't want to wait, you can use ```env.reset(force_dense_logging=True)``` to tell the environment to create a dense log for the upcoming episode.

In [7]:
#display(dense_logs)
display(dummy_env.env.metrics)
#print(env_obj.env.get_agent('p').action_dim)

{'social/productivity': 12128.191460713615,
 'social/equality': 0.7325431219879898,
 'social_welfare/coin_eq_times_productivity': 987.1581374110255,
 'social_welfare/inv_income_weighted_utility': 549.9509472485659,
 'endow/avg_agent/Coin': 1347.5768289681794,
 'endogenous/avg_agent/Labor': 47.55555555555556,
 'util/avg_agent': 931.2636428427551,
 'endow/p/Coin': 0,
 'util/p': 549.9509472485659,
 'PeriodicTax/avg_bracket_rate/000': 0.30874504675364256,
 'PeriodicTax/bracket_occupancy/000': 0.0,
 'PeriodicTax/avg_bracket_rate/097': 0.6991362411273508,
 'PeriodicTax/bracket_occupancy/097': 0.2222222222222222,
 'PeriodicTax/avg_bracket_rate/394': 0.7164617869717881,
 'PeriodicTax/bracket_occupancy/394': 0.3333333333333333,
 'PeriodicTax/avg_bracket_rate/842': 0.6635080665165121,
 'PeriodicTax/bracket_occupancy/842': 0.1111111111111111,
 'PeriodicTax/avg_bracket_rate/1607': 0.5761334259737049,
 'PeriodicTax/bracket_occupancy/1607': 0.2222222222222222,
 'PeriodicTax/avg_bracket_rate/2041': 0

In [5]:
# Shutdown Ray after use
ray.shutdown()

And that's it for now. See you in the [next](https://github.com/salesforce/ai-economist/blob/master/tutorials/two_level_curriculum_learning_with_rllib.md) tutorial :)